In [1]:
from os.path import join, expanduser
DATA_ROOT = join(expanduser("~"), "ACCENT_CONVERSION_DATASET")
!ls $DATA_ROOT
%pylab inline

cmu_us_awb_arctic(modified)   cmu_us_ksp_arctic
cmu_us_bdl_arctic	      cmu_us_ksp_arctic(fucked_up)
cmu_us_bdl_arctic(fucked_up)
Populating the interactive namespace from numpy and matplotlib


In [3]:
from keras.models import Sequential
from keras.layers import Dropout, Dense, BatchNormalization, Activation
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam, SGD, RMSprop

import numpy as np
import matplotlib.pyplot as plt
import librosa

In [3]:
from nnmnkwii.datasets import PaddedFileSourceDataset
from nnmnkwii.datasets.cmu_arctic import CMUArcticWavFileDataSource
from nnmnkwii.preprocessing.alignment import DTWAligner
from nnmnkwii.preprocessing import trim_zeros_frames, remove_zeros_frames, delta_features
from nnmnkwii.util import apply_each2d_trim
from nnmnkwii.metrics import melcd
from nnmnkwii.baseline.gmm import MLPG

from os.path import basename, splitext
import sys
import time

import numpy as np
from scipy.io import wavfile
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import train_test_split
import pyworld
import pysptk
from pysptk.synthesis import MLSADF, Synthesizer
import librosa
import librosa.display
import IPython
from IPython.display import Audio

In [4]:
fs = 16000
fftlen = pyworld.get_cheaptrick_fft_size(fs)
alpha = pysptk.util.mcepalpha(fs)
order = 24
frame_period = 5
hop_length = int(fs * (frame_period * 0.001))
max_files = 100 # number of utterances to be used.
test_size = 0.03
use_delta = True

if use_delta:
    windows = [
        (0, 0, np.array([1.0])),
        (1, 1, np.array([-0.5, 0.0, 0.5])),
        (1, 1, np.array([1.0, -2.0, 1.0])),
    ]
else:
    windows = [
        (0, 0, np.array([1.0])),
    ]

In [5]:
class MyFileDataSource(CMUArcticWavFileDataSource):
    def __init__(self, *args, **kwargs):
        super(MyFileDataSource, self).__init__(*args, **kwargs)
        self.test_paths = None

    def collect_files(self):
        paths = super(
            MyFileDataSource, self).collect_files()
        paths_train, paths_test = train_test_split(
            paths, test_size=test_size, random_state=1234)

        # keep paths for later testing
        self.test_paths = paths_test

        return paths_train

    def collect_features(self, path):
        fs, x = wavfile.read(path)
        x = x.astype(np.float64)
        f0, timeaxis = pyworld.dio(x, fs, frame_period=frame_period)
        f0 = pyworld.stonemask(x, f0, timeaxis, fs)
        spectrogram = pyworld.cheaptrick(x, f0, timeaxis, fs)
        spectrogram = trim_zeros_frames(spectrogram)
        return spectrogram

In [6]:
clb_source = MyFileDataSource(data_root=DATA_ROOT,
                                         speakers=["bdl"], max_files=max_files)
slt_source = MyFileDataSource(data_root=DATA_ROOT,
                                         speakers=["ksp"], max_files=max_files)

In [36]:
import kapre
from kapre.time_frequency import Melspectrogram
from keras.layers import Bidirectional, PReLU, LSTM, GRU

In [44]:
model = Sequential()
#model.add(Melspectrogram(sr = 16000, n_mels = 80, fmin = 0, fmax = 8000, power_melgram = 2.0, return_decibel_melgram = True,
                        #trainable_fb = False, htk = False, norm = 1, name = 'melgram', input_shape = (1, 16000)))
model.add(GRU(80, input_shape = (80, 1000), return_sequences = True))
model.add(Dense(128))
model.add(PReLU())
model.add(Bidirectional(LSTM(100, return_sequences = True)))
model.add(Bidirectional(LSTM(100, return_sequences = True)))
model.add(Dense(80))

In [45]:
optimizer = Adam()
model.compile(optimizer = optimizer, loss = 'mean_squared_error', metrics = ['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_7 (GRU)                  (None, 80, 80)            259440    
_________________________________________________________________
dense_24 (Dense)             (None, 80, 128)           10368     
_________________________________________________________________
p_re_lu_18 (PReLU)           (None, 80, 128)           10240     
_________________________________________________________________
bidirectional_21 (Bidirectio (None, 80, 200)           183200    
_________________________________________________________________
bidirectional_22 (Bidirectio (None, 80, 200)           240800    
_________________________________________________________________
dense_25 (Dense)             (None, 80, 80)            16080     
Total params: 720,128
Trainable params: 720,128
Non-trainable params: 0
_________________________________________________________________
